<a href="https://colab.research.google.com/github/larguesa/FRL-CD-AM2/blob/main/FRL_CD_AM2_04_MNIST_com_input_e_plot_da_RNA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importações necessárias
import tensorflow as tf
import numpy as np
from IPython.display import display, HTML, Javascript, clear_output
import plotly.graph_objects as go
from google.colab import output

# Carregar e preparar o dataset MNIST (subconjunto para acelerar)
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

#--INÍCIO DO bloco que diminui a quantidade de amostras para aceletar (se remover, melhora a precisão)--
# train_size = int(0.1 * len(x_train))
# test_size = int(0.1 * len(x_test))
# x_train, y_train = x_train[:train_size], y_train[:train_size]
# x_test, y_test = x_test[:test_size], y_test[:test_size]
#--FIM DO bloco que diminui a quantidade de amostras para aceletar--

# Carregar e preparar o dataset MNIST
x_train = x_train.astype('float32') / 255.0  # Normalizar para [0, 1]
x_test = x_test.astype('float32') / 255.0
x_train = x_train.reshape(-1, 28, 28, 1)  # Adicionar canal para CNN
x_test = x_test.reshape(-1, 28, 28, 1)

# Definir o modelo SIMPLES usando a API funcional
# inputs = tf.keras.Input(shape=(28, 28, 1))
# x = tf.keras.layers.Conv2D(16, (3, 3), activation='relu')(inputs)
# x = tf.keras.layers.MaxPooling2D((2, 2))(x)
# x = tf.keras.layers.Flatten()(x)
# dense_64 = tf.keras.layers.Dense(64, activation='relu', name='dense_64')(x)
# outputs = tf.keras.layers.Dense(10, activation='softmax')(dense_64)

# Definir o modelo MAIS COMPLEXO usando a API funcional
inputs = tf.keras.Input(shape=(28, 28, 1))
x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')(inputs)  # Aumentar filtros para 32
x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(x)       # Adicionar outra camada Conv2D
x = tf.keras.layers.MaxPooling2D((2, 2))(x)
x = tf.keras.layers.Dropout(0.25)(x)                                # Adicionar Dropout para regularização
x = tf.keras.layers.Flatten()(x)
dense_64 = tf.keras.layers.Dense(128, activation='relu', name='dense_64')(x)  # Aumentar para 128 neurônios
x = tf.keras.layers.Dropout(0.5)(x)                                 # Mais Dropout
outputs = tf.keras.layers.Dense(10, activation='softmax')(dense_64)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

# Compilar e treinar o modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
#epochs=3. Se aumentar, treinamento fica mais preciso
model.fit(x_train, y_train, epochs=3, batch_size=128, validation_data=(x_test, y_test), verbose=1)

# Modelo para obter ativações da camada dense_64
activation_model = tf.keras.Model(inputs=model.input, outputs=model.get_layer('dense_64').output)

# Interface HTML com canvas, botões e persistência do desenho
html_code = """
<canvas id="canvas" width="280" height="280" style="border:1px solid black;"></canvas>
<br>
<button onclick="clearCanvas()">Limpar</button>
<button onclick="processDrawing()">Prever</button>
<div id="debug" style="color: red;"></div>
<script>
  var canvas = document.getElementById('canvas');
  var ctx = canvas.getContext('2d');
  ctx.fillStyle = "white";
  ctx.fillRect(0, 0, canvas.width, canvas.height);
  var drawing = false;
  var lastImage = null;  // Armazenar o estado do canvas

  canvas.addEventListener('mousedown', function(e) { drawing = true; draw(e); });
  canvas.addEventListener('mouseup', function() { drawing = false; ctx.beginPath(); });
  canvas.addEventListener('mousemove', draw);

  function draw(e) {
    if (!drawing) return;
    ctx.lineWidth = 10;
    ctx.lineCap = 'round';
    ctx.strokeStyle = 'black';
    ctx.lineTo(e.offsetX, e.offsetY);
    ctx.stroke();
    ctx.beginPath();
    ctx.moveTo(e.offsetX, e.offsetY);
  }

  function clearCanvas() {
    ctx.fillStyle = "white";
    ctx.fillRect(0, 0, canvas.width, canvas.height);
    lastImage = null;  // Resetar o estado salvo
    document.getElementById('debug').innerText = 'Canvas limpo';
  }

  function saveCanvas() {
    lastImage = canvas.toDataURL();  // Salvar o estado como base64
  }

  function restoreCanvas() {
    if (lastImage) {
      var img = new Image();
      img.src = lastImage;
      img.onload = function() {
        ctx.clearRect(0, 0, canvas.width, canvas.height);
        ctx.drawImage(img, 0, 0);
      };
    }
  }

  function processDrawing() {
    document.getElementById('debug').innerText = 'Processando...';
    saveCanvas();  // Salvar o estado atual do canvas
    var imgData = ctx.getImageData(0, 0, canvas.width, canvas.height);
    var data = imgData.data;
    var grayscale = [];
    for (var i = 0; i < data.length; i += 4) {
      var avg = (data[i] + data[i+1] + data[i+2]) / 3;
      grayscale.push(avg / 255.0);  // Normalizar para [0, 1]
    }
    // Redimensionar para 28x28 diretamente no JavaScript
    var resized = [];
    for (var y = 0; y < 28; y++) {
      for (var x = 0; x < 28; x++) {
        var idx = Math.floor(y * 10) * 280 + Math.floor(x * 10);  // Escalar 280x280 para 28x28
        resized.push(grayscale[idx]);
      }
    }
    document.getElementById('debug').innerText = 'Enviando para Python... Tamanho: ' + resized.length;
    google.colab.kernel.invokeFunction('notebook.processDrawing', [resized], {},
      function() { document.getElementById('debug').innerText = 'Sucesso!'; },
      function(error) { document.getElementById('debug').innerText = 'Erro: ' + error; }
    );
  }
</script>
"""
display(HTML(html_code))

# Função para processar o desenho e exibir ativações em 3D
def process_drawing(grayscale):
    # Limpar a saída anterior
    clear_output(wait=True)

    # Redisplay o canvas e restaurar o desenho
    display(HTML(html_code))
    display(Javascript('restoreCanvas();'))  # Chamar a função para restaurar o canvas

    # Converter o desenho para formato MNIST (28x28)
    img = np.array(grayscale).reshape(1, 28, 28, 1)
    img = 1 - img  # Inverter: branco (0) vira fundo, preto (1) vira dígito

    # Prever o dígito
    prediction = model.predict(img)
    digit = np.argmax(prediction)
    print(f"Dígito previsto: {digit}")

    # Obter ativações da camada Dense
    activations = activation_model.predict(img)[0]

    # Criar visualização 3D com Plotly
    x = np.arange(len(activations))
    y = np.zeros(len(activations))
    z = activations

    fig = go.Figure(data=[go.Scatter3d(
        x=x, y=y, z=z, mode='markers+lines',
        marker=dict(size=5, color=z, colorscale='Viridis'),
        line=dict(width=2)
    )])
    fig.update_layout(
        title="Ativações dos Neurônios na Camada Dense (64 unidades)",
        scene=dict(xaxis_title='Neurônio', yaxis_title='Eixo Y', zaxis_title='Ativação')
    )
    fig.show()

# Registrar a função no kernel do Colab
output.register_callback('notebook.processDrawing', process_drawing)

<IPython.core.display.Javascript object>

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Dígito previsto: 3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
